<a href="https://colab.research.google.com/github/ssunsonic/Data-Science-Fundamentals/blob/main/Pantry_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive for Filepath

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PIP Install Packages

In [2]:
!pip install tabula-py
!pip install df2gspread
# !pip install PyMuPDF
# !pip install pdfminer
# !pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.2 MB/s eta 0:00:00
  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11931 sha256=90597032ac6efbf6082c4de42410946764c4ddac0e6c99c15769c163b81d0545
  Stored in directory: /root/.cache/pip/wheels/80/2f/29/b15c01e2e9fbfaae81e5f9e8a05735954fc125b7cd9f1423a5
Successfully built df2gspread
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

## Import PDF Scraping Packages

In [3]:
# from PyPDF2 import PdfReader
import tabula
import pandas as pd
import glob
import re # regular expr
import numpy as np
import gspread
import df2gspread as d2g

## Using Tabula to Get PDF Paths

In [4]:
# Testing for ALL pdfs under directory
pdf_dir = '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/'

pdf_files = glob.glob("%s/*.pdf" % pdf_dir)
len(pdf_files) # check how many PDFs are there and cross validate
pdf_files

['/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-534919.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-534999.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535064.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535169.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535209.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535306.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535361.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535439.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535515.pdf',
 '/content/drive/Shared drives/Pantry Tech Team/DS Projects/YFB Invoices 23-24/Invoice-535542.pdf',


## Using Paths to Scrape Donated_FA and Local Donations Data

In [5]:
df = []

def scrapepdf(filepaths):

  for file in filepaths: # filepaths for PDFs under directory
    pdf = tabula.read_pdf(file, lattice = True, pages = 'all') # returns a list of dataframes
    data = pd.concat(pdf) # combine all dfs in the list
    data.columns = [column.replace('\r', ' ') for column in data.columns] # cleaning column names
    data = data[['Product', 'Storage', 'Qty', 'UOM', 'Unit Wgt', 'Wgt Ext', 'Price',
       'Price Ext', 'Service Fee', 'Service Fee Ext', 'Total']].dropna(how = 'all').reset_index(drop = True) # select only rows w/ these columns
    df.append(data)

  return df

In [6]:
dummy = scrapepdf(pdf_files)

## Test Clean_DF, How to Get Data only From Donated and Local

In [174]:
# remove certains rows after nth occurrence of NA for data frames

def cleanpdfs(data):

  for i in range(len(data)):
    try: # if keyword exists, we will slice the df at a target index
      idx = data[i][(data[i]['Product'] == 'Purchased') | (data[i]['Product'] == 'State-CalFoods') | (data[i]['Product'] == 'Totals')].index[0] # given column 'Product' for data[i], find index at which keywords appear
      data[i] = data[i].iloc[:idx] # filter data based on keyword
    except IndexError:
      pass

  return data

In [175]:
test = cleanpdfs(dummy)

In [176]:
# remove any NAs
new = [test[i].replace(np.nan, ' ', regex = True) for i in range(len(test))]

In [177]:
new[0]

,Product,Storage,Qty,UOM,Unit Wgt,Wgt Ext,Price,Price Ext,Service Fee,Service Fee Ext,Total
0,Donated FA,,,,,,,,,,
1,700013\rFA Bread-2224\rPackaging:,Dry,185.0,LB,1.0,185.0,$0.000,$0.00,$0.190,$35.15,$35.15
2,700007\rDairy - FA-466,Fridge,180.0,LB,1.0,180.0,$0.000,$0.00,$0.190,$34.20,$34.20
3,700971\rEggs--FA--560,Fridge,120.0,LB,1.0,120.0,$0.000,$0.00,$0.190,$22.80,$22.80
4,702488\rFiber One Fudge\rChocolate Cookie bars...,Dry,2.0,Case,7.0,14.0,$0.000,$0.00,$0.190,$2.66,$2.66
5,702462\rGraham Crackers\rPackaging: 12/14.4oz,Dry,2.0,Case,13.0,26.0,$0.000,$0.00,$0.190,$4.94,$4.94
6,702494\rKeto Friendly Cinnamon\rCranberry Almo...,Dry,1.0,Case,5.0,5.0,$0.000,$0.00,$0.190,$0.95,$0.95
7,"702503\rKettle Chips, Salt &\rVinegar\rPackagi...",Dry,3.0,Case,5.0,15.0,$0.000,$0.00,$0.190,$2.85,$2.85
8,700067\rMac and Cheese\rPackaging: Raley's 48-...,Dry,1.0,Case,28.0,28.0,$0.000,$0.00,$0.190,$5.32,$5.32
9,702449\rOrganic Vegetable Broth\rPackaging: 6/...,Dry,2.0,Case,13.0,26.0,$0.000,$0.00,$0.190,$4.94,$4.94


## Export to Google Sheets

In [11]:
from google.colab import auth
from google.auth import default

By default, gspread will look for OAuth credentials on your drive, but in Google Colab you can authenticate without creating new credentials. Google will ask for permission to access your Google Drive to access spreadsheets.

In [12]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [13]:
worksheet = gc.open("pantry-data") # open new empty worksheet and name it
# sheet1 = worksheet.sheet1

### Testing for Retrieving Product Categories from Sheets

In [14]:
products = worksheet.values_get(range = "A1:B39")

In [15]:
product_data = pd.DataFrame(products['values'])
# USE once, rename columns to Products and Type, drop first row containing these words
product_data = product_data.rename(columns = product_data.iloc[0]).drop(product_data.index[0])

In [16]:
product_data

,Product,Type
1,700013 FA Bread-2224,Baked
2,700007 Dairy - FA-466,Dairy
3,700971 Eggs--FA--560,Produce
4,702488 Fiber One Fudge Chocolate Cookie bars,Snacks
5,702462 Graham Crackers,Snacks
6,702494 Keto Friendly Cinnamon Cranberry Almond...,Packaged
7,"702503 Kettle Chips, Salt & Vinegar",Snacks
8,700067 Mac and Cheese,Packaged
9,702449 Organic Vegetable Broth,Packaged
10,702210 Pinto Beans,Produce


### Adding Categorical Column through Unique Product ID Matching

In [269]:
# unique product IDs for products manually entered in sheets
product_ids = [re.findall('^\d{0,10}', str(x)) for x in product_data['Product']]
product_ids

[['700013'],
 ['700007'],
 ['700971'],
 ['702488'],
 ['702462'],
 ['702494'],
 ['702503'],
 ['700067'],
 ['702449'],
 ['702210'],
 ['700925'],
 ['702428'],
 ['701689'],
 ['700004'],
 ['702296'],
 ['701157'],
 ['702530'],
 ['702390'],
 ['702529'],
 ['700954'],
 ['702295'],
 ['702394'],
 ['702401'],
 ['702353'],
 ['701383'],
 ['702411'],
 ['702345'],
 ['702454'],
 ['702384'],
 ['702554'],
 ['702531'],
 ['702535'],
 ['702547'],
 ['702261'],
 ['702541'],
 ['702456'],
 ['702028'],
 ['702557']]

In [77]:
# separating data frame into products vs product categories
product_check = list(product_data['Product'])
product_check = [element.strip() for element in product_check]
product_type = list(product_data['Type'])

In [40]:
# verification of data retrieval
len(product_check) == len(product_type)

True

In [268]:
# used for matching
ASSIGNMENT = list(list(t) for t in zip(product_ids, product_type))
ASSIGNMENT

[[['700013'], 'Baked'],
 [['700007'], 'Dairy'],
 [['700971'], 'Produce'],
 [['702488'], 'Snacks'],
 [['702462'], 'Snacks'],
 [['702494'], 'Packaged'],
 [['702503'], 'Snacks'],
 [['700067'], 'Packaged'],
 [['702449'], 'Packaged'],
 [['702210'], 'Produce'],
 [['700925'], 'Snacks'],
 [['702428'], 'Canned'],
 [['701689'], 'Packaged'],
 [['700004'], 'Produce'],
 [['702296'], 'Canned'],
 [['701157'], 'Canned'],
 [['702530'], 'Snacks'],
 [['702390'], 'Canned'],
 [['702529'], 'Packaged'],
 [['700954'], 'Canned'],
 [['702295'], 'Canned'],
 [['702394'], 'Canned'],
 [['702401'], 'Packaged'],
 [['702353'], 'Canned'],
 [['701383'], 'Produce'],
 [['702411'], 'Packaged'],
 [['702345'], 'Packaged'],
 [['702454'], 'Wipes'],
 [['702384'], 'Packaged'],
 [['702554'], 'Bagged'],
 [['702531'], 'Snacks'],
 [['702535'], 'Packaged'],
 [['702547'], 'Packaged'],
 [['702261'], 'Snacks'],
 [['702541'], 'Snacks'],
 [['702456'], 'Snacks'],
 [['702028'], 'Produce'],
 [['702557'], 'Snacks']]

## Function for Categorizing Food Products

In [263]:
# Working on Product categorization using regex

def categories(data):

  categories = []

  for k in range(len(data)):
    id = re.findall('^\d{0,10}', data['Product'][k])
    if id in product_ids:
      index = product_ids.index(id)
      categories.append(ASSIGNMENT[index][1])
    else:
      categories.append(' ')

  data["Category"] = categories
  return data

In [264]:
export_data = [categories(df) for df in new]

## Add Local + Donated FA DFS to Different Sheets

Invoice Shipment Dates

In [17]:
dates = []

def invoice_dates(pdf_files):

  for i in range(len(pdf_files)):
    pdf = tabula.read_pdf(pdf_files[i], lattice = True, pages = 1)
    dates.append(pdf[1]['Pickup/Ship Date'][0])

  return dates

In [18]:
pdf_dates = invoice_dates(pdf_files)

Add to Google Sheets

In [19]:
def pdf_to_sheet(cleandf):

  for i in range(len(cleandf)):
    # get df at index i
    df = cleandf[i]

    # create a new sheet
    new_sheet_name = f"{pdf_dates[i]}" # from Invoice Shipment Dates code chunk

    # regex to check if invoice alr exists, do not add
    # only add new invoices if found
    if new_sheet_name in re.findall('\d{0,2}\/\d{0,2}\/\d{1,4}', str([worksheet.get_worksheet(i+1)])):
      pass
    else:
      new_sheet = worksheet.add_worksheet(title=new_sheet_name, rows=df.shape[0], cols=df.shape[1])

      # convert df to list of lists
      data = [df.columns.tolist()] + df.fillna(-1).values.tolist()

      # update the newly created sheet with the data
      new_sheet.update(data)

In [266]:
pdf_to_sheet(export_data) # call this to add to sheets